In [87]:
from selenium import webdriver
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from pymongo import MongoClient
from selenium.webdriver import Firefox
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

import pandas as pd
import glob
import json
import time
import re
import os

## Getting product URL's & downloading those html pages

In [5]:
# Using Firefox
base_url='https://www.amazon.com'
url='https://www.amazon.com/s?k=laptop'

## >>>>PATH NEEDS TO BE CHANGED HERE<<<<<
executable_path='/Users/aravindsampath/Desktop/Winter Quarter/BAX 422 Data Design & Representation/Assignments/Final Project/geckodriver'
options = Options()
options.headless = True
driver = webdriver.Firefox(executable_path=executable_path,options=options) #,options=options

driver.implicitly_wait(10)
driver.set_script_timeout(120)
driver.set_page_load_timeout(30)
#load
driver.get(url)
time.sleep(5)

page_source = driver.page_source

# Parse the HTML content of the page using BeautifulSoup
soup = BeautifulSoup(page_source, "html.parser")

# Find the "Highly rated" section of the page
highly_rated = soup.find("div", {"class": "s-include-content-margin s-border-bottom s-border-top-overlap s-widget-padding-bottom"})
highly_rated_data=highly_rated.find('span',{'data-component-type':'s-searchgrid-carousel'})

laptops_1 = highly_rated.find_all("li", {"class": "a-carousel-card"}) #backup is laptops variable

laptops_info = []
for laptop in laptops_1:
    a_tag = laptop.find("a", {"class": "a-link-normal s-no-outline"})
    if a_tag:
        laptop_url = a_tag.get("href")
        if laptop_url.startswith('/'):
            laptop_url = urljoin(base_url, laptop_url)
        laptops_info.append(laptop_url)
        print(f"Added laptop {laptop_url} to laptops_info.")
    else:
        print("No a_tag found for laptop.")
    

for i, product_url in enumerate(laptops_info):
    try:
        driver.get(product_url)  
        time.sleep(10)
        # Save the HTML content of the response to a local file
        filename = f"product_{i+1}.htm"
        with open(filename, "wb") as f:
            f.write(driver.page_source.encode('utf-8'))
        print(f"Saved product {i+1}")
    except Exception as e:
        print(f"Error occurred while processing product {i+1}: {str(e)}")


/var/folders/_h/288b29z5047d1xgptl1cp0z40000gn/T/ipykernel_38992/3085368168.py:9: DeprecationWarning: headless property is deprecated, instead use add_argument('-headless')
  options.headless = True
/var/folders/_h/288b29z5047d1xgptl1cp0z40000gn/T/ipykernel_38992/3085368168.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=executable_path,options=options) #,options=options


Added laptop https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_search_thematic_aps_sr_pg1_1?ie=UTF8&adId=A02523911JS5II9M1YXMK&qualifier=1678675412&id=5711580818133526&widgetName=sp_search_thematic&url=%2FHP-Pavilion-Quad-core-Processor-Bluetooth%2Fdp%2FB0B3L48YH9%2Fref%3Dsxin_14_pa_sp_search_thematic_sspa%3Fcontent-id%3Damzn1.sym.c3a4cfcc-a011-4761-9276-83be9bd7db94%253Aamzn1.sym.c3a4cfcc-a011-4761-9276-83be9bd7db94%26cv_ct_cx%3Dlaptop%26keywords%3Dlaptop%26pd_rd_i%3DB0B3L48YH9%26pd_rd_r%3D925ad1b5-2f93-4eb6-a1bd-ba086f4d1099%26pd_rd_w%3DJ1faj%26pd_rd_wg%3DUQsLP%26pf_rd_p%3Dc3a4cfcc-a011-4761-9276-83be9bd7db94%26pf_rd_r%3DVQTFP33VQDKWZ0674X9P%26qid%3D1678675412%26sbo%3DRZvfv%252F%252FHxDF%252BO5021pAnSA%253D%253D%26sr%3D1-1-1fa5bb29-8495-43da-8896-6cda87f207d0-spons%26psc%3D1 to laptops_info.
Added laptop https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_search_thematic_aps_sr_pg1_2?ie=UTF8&adId=A05122063DLDB0NKNQUUK&qualifier=1678675412&id=57115808

## Getting Review URL's

In [6]:
# Main Review URLs
review_urls=[]
for i, product_url in enumerate(laptops_info):
    try:
        driver.get(product_url)  
        time.sleep(10)
        # Save the HTML content of the response to a local file
        reviews_source=driver.page_source
        reviews_soup = BeautifulSoup(reviews_source, "html.parser")
        rev=reviews_soup.find('a',{'data-hook':'see-all-reviews-link-foot'})['href']
        if rev.startswith('/'):
            rev2 = urljoin(base_url, rev)
        review_urls.append(rev2)
        
    except Exception as e:
        print(f"Error occurred while processing product {i}")


['https://www.amazon.com/HP-Pavilion-Quad-core-Processor-Bluetooth/product-reviews/B0B3L48YH9/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews']
['https://www.amazon.com/HP-Pavilion-Quad-core-Processor-Bluetooth/product-reviews/B0B3L48YH9/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews', 'https://www.amazon.com/HP-Laptop-Quad-core-Processor-Bluetooth/product-reviews/B0BPTKJ5JC/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews']
['https://www.amazon.com/HP-Pavilion-Quad-core-Processor-Bluetooth/product-reviews/B0B3L48YH9/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews', 'https://www.amazon.com/HP-Laptop-Quad-core-Processor-Bluetooth/product-reviews/B0BPTKJ5JC/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews', 'https://www.amazon.com/Acer-AV15-51-7617-i7-1195G7-Graphics-Materials/product-reviews/B09R63Z5L7/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews']
['https://www.amazon.com/HP-Pavilion-Quad-core-Processor-Blu

## Getting all reivew pages urls and then downloading the htmls

In [31]:

# create a folder to store the HTML files
if not os.path.exists('html_pages'):
    os.makedirs('html_pages')


# scrape all review pages for each main review URL and save as HTML files
for i, rev_url in enumerate(review_urls):
    j = 1
    product = f'product_{i+1}'
    while True:
        try:
            url = f'{rev_url}&pageNumber={j}'
            driver.get(url)
            time.sleep(10)
            reviews_source = driver.page_source
            with open(f'html_pages/{product}_page{j}.html', 'w', encoding='utf-8') as f:
                f.write(reviews_source)
            reviews_soup = BeautifulSoup(reviews_source, "html.parser")
            reviews = reviews_soup.find_all('div', {'id': 'cm_cr-review_list'})
            if not reviews or not reviews[0].find_all('div', {'data-hook': 'review'}):
                break
            j += 1
        except Exception as e:
            print(f"Error occurred while processing product {i+1}, page {j}")
            break


## Fetching Laptop Information from main URL's

In [66]:
html_files = [file for file in sorted(os.listdir()) if file.endswith('.htm')]

   
# process each HTML file and extract the laptop information
laptops_info = []
for i, file in enumerate(html_files):
   
    try:
        with open(file, 'r', encoding='utf-8') as f:
            content = f.read()
        soup = BeautifulSoup(content, 'html.parser')
        try:
            Product_Title = soup.find('span', {'id': 'productTitle'}).text.strip()
        except:
            Product_Title = ''

        # extract the laptop information section
        try:
            info_section = soup.find('div', {'id': 'productOverview_feature_div'})
        except:
            info_section = None
            

        # create a dictionary to store the extracted information
        laptop_info = {'Product_Title': Product_Title}

        # extract individual attributes from the laptop information section
        if info_section:
            for item in info_section.find_all('tr'):
                try:
                    attribute = item.find('td', {'class': 'a-span3'}).text.strip()
                    value = item.find('td', {'class': 'a-span9'}).text.strip()
                    laptop_info[attribute] = value
                except:
                    pass

        # extract the price information
        try:
            price_section = soup.find('span', {'class': 'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'})
            price = price_section.find('span', {'class': 'a-offscreen'}).text.strip()
            laptop_info['Price'] = price
        except:
            laptop_info['Price'] = ''

        # extract the rating information
        try:
            rating_section = soup.find('div', {'id': 'averageCustomerReviews'})
            overall_rating = rating_section.find('span', {'class': 'a-icon-alt'}).text.strip()
            laptop_info['overall_rating'] = overall_rating
        except:
            laptop_info['overall_rating'] = ''
        
        laptop_info['product'] = file.split('.')[0]
        
        laptops_info.append(laptop_info)
        print(f"Extracted laptop information from file {file}")
    except Exception as e:
        print(f"Error occurred while processing file {file}: {str(e)}")

# print the extracted laptop information
print(laptops_info[0])

Extracted laptop information from file product_1.htm
Extracted laptop information from file product_2.htm
Extracted laptop information from file product_3.htm
Extracted laptop information from file product_4.htm
Extracted laptop information from file product_5.htm
{'Product_Title': 'HP 2022 Newest Pavilion 15.6" HD Laptop, Intel Quad-core Pentium Processor, 16GB RAM, 1TB SSD, 11 Hr Battry Life, Intel UHD Graphics, HD Webcam, Bluetooth, HDMI, USB Type-C, Scarlet Red, Win 11', 'Brand': 'HP', 'Model Name': 'Pavilion', 'Screen Size': '15.6 Inches', 'Hard Disk Size': '1 TB', 'CPU Model': 'Pentium N5000', 'Ram Memory Installed Size': '16 GB', 'Operating System': 'Windows 11', 'Graphics Card Description': 'Integrated', 'Graphics Coprocessor': 'Intel UHD Graphics', 'CPU Speed': '2.7 GHz', 'Price': '$469.00', 'overall_rating': '4.3 out of 5 stars', 'product': 'product_1'}


In [79]:
Laptop_Information = []
for laptop_info in laptops_info:
    laptop_information = {
        'Product': laptop_info.get('product', ''),
        'Product_Title': laptop_info.get('Product_Title', ''),
        'Price': laptop_info.get('Price', ''),
        'Brand': laptop_info.get('Brand', ''),
        'Screen_Size': laptop_info.get('Screen Size', ''),
        'Hard_Disk_Size': laptop_info.get('Hard Disk Size', laptop_info.get('Memory Storage Capacity', '')),
        'CPU_Model': laptop_info.get('CPU Model', ''),
        'RAM_memory_installed_size': laptop_info.get('Ram Memory Installed Size', ''),
        'Operating_System': laptop_info.get('Operating System', ''),
        'Graphics_Card_Description': laptop_info.get('Graphics Card Description', laptop_info.get('Card Description','')),
        'Graphics_Coprocessor': laptop_info.get('Graphics Coprocessor', ''),
        'CPU_speed': laptop_info.get('CPU Speed', ''),
        'Overall_Rating': laptop_info.get('overall_rating', '')
    }

    Laptop_Information.append(laptop_information)


In [80]:
Laptop_Information

[{'Product': 'product_1',
  'Product_Title': 'HP 2022 Newest Pavilion 15.6" HD Laptop, Intel Quad-core Pentium Processor, 16GB RAM, 1TB SSD, 11 Hr Battry Life, Intel UHD Graphics, HD Webcam, Bluetooth, HDMI, USB Type-C, Scarlet Red, Win 11',
  'Price': '$469.00',
  'Brand': 'HP',
  'Screen_Size': '15.6 Inches',
  'Hard_Disk_Size': '1 TB',
  'CPU_Model': 'Pentium N5000',
  'RAM_memory_installed_size': '16 GB',
  'Operating_System': 'Windows 11',
  'Graphics_Card_Description': 'Integrated',
  'Graphics_Coprocessor': 'Intel UHD Graphics',
  'CPU_speed': '2.7 GHz',
  'Overall_Rating': '4.3 out of 5 stars'},
 {'Product': 'product_2',
  'Product_Title': 'HP 15.6" Laptop with Intel 4-core CPU, 15.6" HD LED Display, Intel Quad-core Processor, Bluetooth and Wi-Fi, HDMI, Long Battery Life, Windows 11 Home in S Mode(16GB RAM | 1TB SSD)',
  'Price': '$499.00',
  'Brand': 'HP',
  'Screen_Size': '15.6 Inches',
  'Hard_Disk_Size': '1 TB',
  'CPU_Model': 'Pentium',
  'RAM_memory_installed_size': '16 G

## Fetching Reivew details from the downloaded review HTML pages

In [88]:

reviews = []
current_product = None
current_review = 0
pattern = r'[^a-zA-Z0-9.,\'\s]'
# regular expression pattern to match all non-alphanumeric characters except for periods, commas, and apostrophes
        

for file_name in sorted(os.listdir('html_pages')):
    # skip files that don't have the expected format
    if not file_name.startswith('product_') or not file_name.endswith('.html'):
        continue
    
    # extract product name and page number from file name
    product, page_num = file_name.split('_page')
    page_num = int(page_num.split('.')[0])
    
    # check if product has changed and reset review count
    if product != current_product:
        current_product = product
        current_review = 0
    
    # parse HTML content using BeautifulSoup
    with open(f'html_pages/{file_name}', 'r', encoding='ISO-8859-1') as f:
        page_source = f.read()
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # extract reviews and ratings
    review_boxes = soup.find_all('div', {'data-hook': 'review'})
    for i, box in enumerate(review_boxes):
        current_review += 1
        review = f'review_{current_review}'
        rating_element = box.find('i', {'data-hook': 'review-star-rating'})
        if rating_element:
            rating = rating_element.text.strip().split(' ')[0]
        else:
            rating = None
        text_element = box.find('span', {'data-hook': 'review-body'})
        if text_element:
            text_1 = text_element.text.strip()
            text = re.sub(pattern, '', text_1)
        else:
            text = None
    
        reviews.append({'product': product, 'review': review, 'rating': rating, 'text': text})

df = pd.DataFrame(reviews, columns=['product', 'review', 'rating', 'text'])


In [89]:
#df.to_csv('reviews.csv', index=False)

In [90]:
Laptop_Reviews = []
for i,j in df.iterrows():
    laptop_review = {
        'Product': j['product'],
        'Review_Number':j['review'],
        'Rating': j['rating'],
        'Text Review': j['text']
    }

    Laptop_Reviews.append(laptop_review)


In [91]:
Laptop_Reviews

[{'Product': 'product_1',
  'Review_Number': 'review_1',
  'Rating': '4.0',
  'Text Review': 'I chose this computer because of the specs. 32G Ram and 1TB SSD.'},
 {'Product': 'product_1',
  'Review_Number': 'review_2',
  'Rating': '4.0',
  'Text Review': 'Amazing for everything except Video Games'},
 {'Product': 'product_1',
  'Review_Number': 'review_3',
  'Rating': '5.0',
  'Text Review': "I bought this as a gift for my husband to use for gaming, then he told me he could use this for work, not something he had ever mentioned because he likes to leave work at work, but as the world gets more computerized a note through email is easier than a hand written note left on a desk that for some reason was never seen. Each day he sees ways it will make little irritating things better, and its making those little irritating things better for me because I don't have to listen to them The laptop is easy to use and for the moment has plenty of memory and as he figures out all the things he can do

## Creating database and collections in MongoDB Instance

In [93]:
# Mongo DB


# connect to the MongoDB server
client = MongoClient('mongodb://localhost:27017/')

# create a database named "amazon_data"
db = client['amazon_highly_rated']

# create a collection named "laptop_details"
laptop_details_collection = db['Laptop_Information']

# create a collection named "laptop_reviews"
laptop_reviews_collection = db['Laptop_Reviews']


## Pushing the Laptop Information into MongoDB

In [94]:
# iterate over the laptop information
for laptop_info in Laptop_Information:
    # insert the laptop information into the "laptop_details" collection
    laptop_details_collection.insert_one(laptop_info)


## Pushing the Laptop Reviews information into MongoDB

In [95]:
# iterate over the laptop reviews
for laptop_review in Laptop_Reviews:
    # insert the laptop Reviews into the "laptop_reviews" collection
    laptop_reviews_collection.insert_one(laptop_review)